In [16]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import numpy as np
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

class AlgebraicSearchEngine:
    def __init__(self,
                 docs : list[str],
                 stemmer=PorterStemmer(),
                 vectorizer=TfidfVectorizer(),
                 stopwords=stopwords.words('english')):

        self.stemmer = stemmer
        self.vectorizer = vectorizer
        self.stopwords = stopwords

        proc_docs = []
        for doc in docs:
            proc_tokens = self.process_doc(doc)
            proc_doc = ' '.join(proc_tokens)
            # Add the stemmed document to the list of stemmed documents
            proc_docs.append(proc_doc)

        # Store the original and processed documents
        self.docs = docs
        # Store the processed document vectors
        self.vecs = self.vectorizer.fit_transform(proc_docs).toarray()

    def process_query(self, query: str) -> list[str]:
        tokens = re.findall(r'\b\w+\b|\(|\)', query)
        tokens = [self.stemmer.stem(token) for token in tokens]
        return tokens

    def process_doc(self, doc : str) -> list[str]:
        # remove punctuation
        doc = re.sub(r'[^\w\s]', '', doc)
        # lower-case
        doc = doc.lower()
        # split the string into words
        words = doc.split()
        # remove stopwords
        words = [word for word in words if word not in self.stopwords]
        # stem each word and join them back into a string
        words = [self.stemmer.stem(word) for word in words]
        return words

    def recursive_search(self, tokens):
        operator = tokens.pop(0).upper()

        if operator not in ['AND', 'OR', 'NOT']:
            raise ValueError(f"Invalid operator {operator}")
        
        operands = []
        while tokens[0] != ')':
            if tokens[0] == '(':
                tokens.pop(0)  # Remove '('
                operands.append(self.recursive_search(tokens))
            else:
                term_vec = self.vectorizer.transform([tokens.pop(0)]).toarray()[0]
                term_scores = self.vecs.dot(term_vec)
                operands.append(term_scores)

        tokens.pop(0)  # Remove ')'
        result = None
        if operator == 'AND':
            result = np.min(np.array(operands), axis=0)
        elif operator == 'OR':
            result = np.max(np.array(operands), axis=0)
        elif operator == 'NOT':
            if len(operands) != 1:
                raise ValueError("NOT operator can only have one operand")
            #result = 1 - operands[0]
            result = operands[0]**0.5
        return result
    
    def search(self, query):
        tokens = self.process_query(query)
        if tokens[0] != '(':
            raise ValueError("Invalid query")
        
        tokens = tokens[1:]

        scores = self.recursive_search(tokens)
        return scores


In [17]:
docs = ["The cat in the hat",
        "This is just a document with no other purpose than to show how the search engine works.",
        "A dog and his boy.",
        "A boy jumps up and down.",
        "The cats are out of the bag.",
        "Dogs and cats, living together.",
        "The quick brown fox jumps over the lazy dog.",
        "Cats, cats, cats, cats, cats, and maybe a dog!",
        "The dog did not bite the cat.",
        "quick dog cat",
        "a quick dog bite a cat",
        "dog cat",
        "quick dog",
        "Dog, dogs, dogs, dogs, dogs! And maybe a cat.",
        "Dog, dogs, dogs! And maybe a cat.",
        "Okay, now is the time, for all the good men, to come to the aid of their country.",
        "Cat cat cat cat cat cat cats cats cats!",
        "cat dog quick",
        "test"]
boolean_search_engine = AlgebraicSearchEngine(
    docs=docs,
    vectorizer=CountVectorizer(binary=True))

boolean_search_engine.process_doc("Dogs and cats, living together!!!")
# Document: "Dogs and cats, living together!!!",
#   boolean_search_engine.process_doc("Dogs and cats, living together!!!")
#   -> pre-processed: ['dogs', 'and', 'cats', 'living', 'together']
#   -> stop-words removed: ['dogs', 'cats', 'living', 'together']
#   -> stemmed words: ['dog', 'cat', 'live', 'togeth']

['dog', 'cat', 'live', 'togeth']

Now we show how a query is processed. Recall the query permits `AND`, `OR`, and
`NOT`, which is sufficient to implement a Boolean algebra. In particular, this
means that given a query, a subset of the documents is denoted by the query.

We imagine that the Boolean algebra is over the powerset of the words in the
all the documents. A query is a special way of specifying a subset of the
powerset of the words in the documents. The subset is specified by the
occurrence of the words in the query. For example, the query `A AND B` denotes
the set of documents that contain both `A` and `B`. The query `A OR B` denotes
the set of documents that contain either `A` or `B`. The query `NOT A` denotes
the set of documents that do not contain `A`. We may combine these to form
complex queries. For example, the query `A AND (B OR NOT C) AND NOT D` denotes the
set of documents that contain `A` and either `B` or not `C` and do not contain
`D`.

A document is said to be relevant to a query if it is in the subset denoted by
the query. The goal of the search engine is to return the documents that are
relevant to the query.

In [18]:
query = "(AND cat (NOT bite) dog)"# (AND dog quick) (NOT (AND boy jump)))"
#query = "(AND cat (NOT dog))"
print(f"Query: {query}")
# Query: (AND cat (NOT (OR dog men))
print(f'Processed Query: {boolean_search_engine.process_query(query)}')
# Processed Query: ['(', 'and', 'cat', '(', 'not', '(', 'or', 'dog', 'men', ')', ')']

Query: (AND cat (NOT bite) dog)
Processed Query: ['(', 'and', 'cat', '(', 'not', 'bite', ')', 'dog', ')']


In [ ]:
query = "(AND cat (NOT bite) dog)"# (AND dog quick) (NOT (AND boy jump)))"
#query = "(AND cat (NOT dog))"
print(f"Query: {query}")
# Query: (AND cat (NOT (OR dog men))
print(f'Processed Query: {boolean_search_engine.process_query(query)}')
# Processed Query: ['(', 'and', 'cat', '(', 'not', '(', 'or', 'dog', 'men', ')', ')']
results = boolean_search_engine.search(query)
print(f"{results=} {results.shape=}")
# sort results by score
results = sorted(enumerate(results), key=lambda x: x[1], reverse=True)
# pretty print the results
for i, doc in enumerate(boolean_search_engine.docs):
    print(f"Document {i}: {doc} => {results[i]}")



results=array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0.]) results.shape=(19,)
Document 0: The cat in the hat => (8, 1.0)
Document 1: This is just a document with no other purpose than to show how the search engine works. => (10, 1.0)
Document 2: A dog and his boy. => (0, 0.0)
Document 3: A boy jumps up and down. => (1, 0.0)
Document 4: The cats are out of the bag. => (2, 0.0)
Document 5: Dogs and cats, living together. => (3, 0.0)
Document 6: The quick brown fox jumps over the lazy dog. => (4, 0.0)
Document 7: Cats, cats, cats, cats, cats, and maybe a dog! => (5, 0.0)
Document 8: The dog did not bite the cat. => (6, 0.0)
Document 9: quick dog cat => (7, 0.0)
Document 10: a quick dog bite a cat => (9, 0.0)
Document 11: dog cat => (11, 0.0)
Document 12: quick dog => (12, 0.0)
Document 13: Dog, dogs, dogs, dogs, dogs! And maybe a cat. => (13, 0.0)
Document 14: Dog, dogs, dogs! And maybe a cat. => (14, 0.0)
Document 15: Okay, now is the time, for al

Now we use TF-IDF to score the words in the text. We will use the TfidfVectorizer from the scikit-learn library to convert the text into a matrix of TF-IDF features. We will then use the cosine similarity to find the similarity between the text and the query.

In [122]:


fuzzy_search_engine =   AlgebraicSearchEngine(docs=docs, vectorizer=TfidfVectorizer())
results = fuzzy_search_engine.search(query)
# pretty print the results
for i, doc in enumerate(fuzzy_search_engine.docs):
    print(f"Document {i+1}: {doc} => {results[i]}")


Document 1: The cat in the hat => 0.0
Document 2: This is just a document with no other purpose than to show how the search engine works. => 0.0
Document 3: A dog and his boy. => 0.0
Document 4: A boy jumps up and down. => 0.0
Document 5: The cats are out of the bag. => 0.0
Document 6: Dogs and cats, living together. => 0.28958595184220776
Document 7: The quick brown fox jumps over the lazy dog. => 0.0
Document 8: Cats, cats, cats, cats, cats, and maybe a dog! => 0.43065672355472245
Document 9: The dog did not bite the cat. => 0.4151629224275721
Document 10: quick dog cat => 0.46832112601474213
Document 11: a quick dog bite a cat => 0.3458313340667656
Document 12: dog cat => 0.7071067811865476
Document 13: quick dog => 0.0
Document 14: Dog, dogs, dogs, dogs, dogs! And maybe a cat. => 0.18546521354288864
Document 15: Dog, dogs, dogs! And maybe a cat. => 0.2765851124973335
Document 16: Okay, now is the time, for all the good men, to come to the aid of their country. => 0.0
Document 17: C